# Text classification using Keras with Neptune tracking
Notebook inspired from https://keras.io/examples/nlp/text_classification_from_scratch/

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/use-cases/nlp/classification/keras/code/keras_nb.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/use-cases/nlp/classification/keras/code/keras_nb.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>

## (Neptune) Install the neptune-notebooks widget (optional)
The neptune-notebooks jupyter extension lets you version, manage and share notebook checkpoints in your projects, without leaving your notebook.  
[Read the docs](https://docs.neptune.ai/integrations-and-supported-tools/ide-and-notebooks/jupyter-lab-and-jupyter-notebook)

## Setup

### Install requirements

In [ ]:
%pip install -U -q neptune[tensorflow-keras] numpy pydot tensorflow graphviz

In [ ]:
import os
import tensorflow as tf

### Create utility functions

In [ ]:
def extract_files(source: str, destination: str) -> None:
    """Extracts files from the source archive to the destination path

    Args:
        source (str): Archive file path
        destination (str): Extract destination path
    """

    import tarfile

    print("Extracting data...")
    with tarfile.open(source) as f:
        f.extractall(destination)


def prep_data(imdb_folder: str, dest_path: str) -> None:
    """Removes unnecessary folders/files and renames source folder

    Args:
        imdb_folder (str): Path of the aclImdb folder
        dest_name (str): Destination folder to which the aclImdb folder has to be renamed to
    """
    import os
    import shutil

    shutil.rmtree(f"{imdb_folder}/train/unsup")
    os.remove(f"{imdb_folder.rsplit('/', maxsplit=1)[0]}/aclImdb_v1.tar.gz")

    if os.path.exists(dest_path):
        shutil.rmtree(dest_path)

    os.rename(imdb_folder, dest_path)
    print(f"{imdb_folder} renamed to {dest_path}")


def custom_standardization(input_data):
    import string
    import re

    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")


def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


def build_model(model_params: dict, data_params: dict):
    """Accepts model and data parameters to build and compile a keras model

    Args:
        model_params (dict): Model parameters
        data_params (dict): Data parameters

    Returns:
        A compiled keras model
    """

    import tensorflow as tf
    from tensorflow.keras import layers

    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(None,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(data_params["max_features"], data_params["embedding_dim"])(inputs)
    x = layers.Dropout(model_params["dropout"])(x)

    # Conv1D + global max pooling
    x = layers.Conv1D(
        data_params["embedding_dim"],
        model_params["kernel_size"],
        padding="valid",
        activation=model_params["activation"],
        strides=model_params["strides"],
    )(x)
    x = layers.Conv1D(
        data_params["embedding_dim"],
        model_params["kernel_size"],
        padding="valid",
        activation=model_params["activation"],
        strides=model_params["strides"],
    )(x)
    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = layers.Dense(data_params["embedding_dim"], activation=model_params["activation"])(x)
    x = layers.Dropout(model_params["dropout"])(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

    keras_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    keras_model.compile(
        loss=model_params["loss"],
        optimizer=model_params["optimizer"],
        metrics=model_params["metrics"],
    )

    return keras_model

### (Neptune) Import Neptune and initialize a project
**A project is a collection of runs, models, and other metadata created by project members.** Typically you should create one project per machine learning task, to make it easy to compare runs that are connected to building certain kinds of ML model.  
[Read the docs](https://docs.neptune.ai/you-should-know/core-concepts#project)

In [ ]:
os.environ["NEPTUNE_PROJECT"] = "common/project-text-classification"

You need a Neptune API token to be able to log to Neptune.
Read how to get and use one [here](https://docs.neptune.ai/setup/setting_api_token/#setting-your-api-token).

**or** 

If you don't have an API token, you can use the `neptune.ANONYMOUS_API_TOKEN` to log to a public project.  
To log anonymously to a public project, set it as your environment variable as below:

```python
os.environ["NEPTUNE_API_TOKEN"] = neptune.ANONYMOUS_API_TOKEN
```

In [ ]:
import neptune

project = neptune.init_project()

## Data preparation
We are using the IMDB sentiment analysis data available at https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz. For the purposes of this demo, we've uploaded this data to S3 at https://neptune-examples.s3.us-east-2.amazonaws.com/data/text-classification/aclImdb_v1.tar.gz and will be downloading it from there.

### (Neptune) Track datasets using Neptune
Neptune lets you track pointers to datasets, models, and other artifacts stored locally or in S3.  
To use this, you will need to have your `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` environment variables set.  
[Read the docs](https://docs.neptune.ai/how-to-guides/data-versioning)

Since this dataset will be used among all the runs in the project, we track it at the project level.
Read more about logging project-level metadata [here](https://docs.neptune.ai/logging/project_metadata/#logging-project-level-metadata).


In [ ]:
# project["keras/data/files"].track_files(
#     "s3://neptune-examples/data/text-classification/aclImdb_v1.tar.gz"
# )
# project.wait()

### (Neptune) Download files from S3 using Neptune
You can also download tracked files from S3 using Neptune, without having to write boilerplate boto3 code.
Read the artifact API reference to know more: https://docs.neptune.ai/api/field_types/#download

In [ ]:
# print("Downloading data...")
# project["keras/data/files"].download("..")

### Prepare data

In [ ]:
extract_files(source="../aclImdb_v1.tar.gz", destination="..")
prep_data(
    imdb_folder="../aclImdb", dest_path="../data"
)  # If you get a permission error here, you can manually rename the `aclImdb` folder to `data`

#### (Neptune) Upload dataset sample to Neptune project
In addition to tracking external files, you can also upload them directly to Neptune.
Such uploaded files can be visualized directly in the Neptune app.  
[Read more here](https://docs.neptune.ai/logging/files/)

In [ ]:
import random

base_namespace = "keras/data/sample/"

project[base_namespace]["train/pos"].upload(
    f"../data/train/pos/{random.choice(os.listdir('../data/train/pos'))}"
)
project[base_namespace]["train/neg"].upload(
    f"../data/train/neg/{random.choice(os.listdir('../data/train/neg'))}"
)
project[base_namespace]["test/pos"].upload(
    f"../data/test/pos/{random.choice(os.listdir('../data/test/pos'))}"
)
project[base_namespace]["test/neg"].upload(
    f"../data/test/neg/{random.choice(os.listdir('../data/test/neg'))}"
)

### (Neptune) Initialize a run
**A run is a namespace inside a project where you log metadata.** Typically, you create a run every time you execute a script that does model training, re-training, or inference.  
[Read the docs](https://docs.neptune.ai/you-should-know/core-concepts#run)

In [ ]:
run = neptune.init_run(
    name="Keras text classification",
    tags=["keras", "notebook"],
    dependencies="requirements.txt",
)

#### (Neptune) Log data metadata to run
You can log nested dictionaries to create custom nested namespaces.  
[Read the docs](https://docs.neptune.ai/logging/methods/#essential-logging-methods)

In [ ]:
data_params = {
    "batch_size": 16,
    "validation_split": 0.2,
    "max_features": 1500,
    "embedding_dim": 128,
    "sequence_length": 1000,
    "seed": 42,
}

In [ ]:
run["data/params"] = data_params

#### # (Neptune) Track dataset at the run-level
We can fetch the dataset from the project metadata and track it at the run level using the `fetch()` method.  
[`fetch()` API reference](https://docs.neptune.ai/api/field_types/#fetch)

In [ ]:
run["data/files"] = project["keras/data/files"].fetch()

### Generate training, validation, and test datasets

In [ ]:
raw_train_ds, raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/train",
    batch_size=data_params["batch_size"],
    validation_split=data_params["validation_split"],
    subset="both",
    seed=data_params["seed"],
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/test", batch_size=data_params["batch_size"]
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

### Clean data

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=data_params["max_features"],
    output_mode="int",
    output_sequence_length=data_params["sequence_length"],
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### Vectorize data

In [ ]:
# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## Modelling

### (Neptune) Register a model and create a new model version
With Neptune's model registry, you can store your ML models in a central location and collaboratively manage their lifecycle.  
[Read the docs](https://docs.neptune.ai/how-to-guides/model-registry)

In [ ]:
from neptune.exceptions import NeptuneModelKeyAlreadyExistsError

project_key = project["sys/id"].fetch()
model_key = "KER"

try:
    model = neptune.init_model(name="keras", key=model_key)
    model.stop()
except NeptuneModelKeyAlreadyExistsError:
    # If it already exists, we don't have to do anything.
    pass

model_version = neptune.init_model_version(model=f"{project_key}-{model_key}", name="keras")

### Build a model

In [ ]:
model_params = {
    "dropout": 0.4,
    "strides": 3,
    "activation": "relu",
    "kernel_size": 3,
    "loss": "binary_crossentropy",
    "optimizer": "adam",
    "metrics": ["accuracy"],
}

In [ ]:
from neptune.utils import stringify_unsupported


model_version["params/model"] = run["training/model/params"] = stringify_unsupported(model_params)
model_version["params/data"] = data_params

In [ ]:
keras_model = build_model(model_params, data_params)

### Train the model

#### (Neptune) Initialize the Neptune callback
The Neptune–Keras integration logs the following metadata automatically:

* Model summary
* Parameters of the optimizer used for training the model
* Parameters passed to Model.fit during the training
* Current learning rate at every epoch
* Hardware consumption and stdout/stderr output during training
* Training code and Git information

Read more about the Neptune–Keras integration here: https://docs.neptune.ai/integrations/keras/

In [ ]:
from neptune.integrations.tensorflow_keras import NeptuneCallback

neptune_callback = NeptuneCallback(run=run, log_model_diagram=False, log_on_batch=True)

In [ ]:
training_params = {
    "epochs": 2,
}

In [ ]:
# Fit the model using the train and test datasets.
keras_model.fit(
    train_ds, validation_data=val_ds, epochs=training_params["epochs"], callbacks=neptune_callback
)
# Training parameters are logged automatically to Neptune

### Evaluate the model

In [ ]:
# We save the accuracy of the  model to be able to evaluate it against the champion model in production later in the notebook
_, curr_model_acc = keras_model.evaluate(test_ds, callbacks=neptune_callback)

### (Neptune) Associate run with model and vice-versa
We can fetch metadata from the run's `sys` namespace and add those to the model_version to be able to link model versions with the runs that created them, and vice-versa.

In [ ]:
run_meta = {
    "id": run["sys/id"].fetch(),
    "name": run["sys/name"].fetch(),
    "url": run.get_url(),
}

print(run_meta)

In [ ]:
model_version["run"] = run_meta

In [ ]:
model_version_meta = {
    "id": model_version["sys/id"].fetch(),
    "name": model_version["sys/name"].fetch(),
    "url": model_version.get_url(),
}

print(model_version_meta)

In [ ]:
run["training/model/meta"] = model_version_meta

### (Neptune) Upload serialized model and model weights to Neptune

In [ ]:
model_version["serialized_model"] = keras_model.to_json()

In [ ]:
keras_model.save_weights("model_weights.h5")
model_version["model_weights"].upload("model_weights.h5")

### (Neptune) Update model stage
We can update the model stage both in the app and through the API.  
[Read the docs](https://docs.neptune.ai/model_registry/managing_stage/)

In [ ]:
model_version.change_stage("staging")

### (Neptune) Wait for all operations to reach with Neptune servers
Since Neptune sends data to servers asynchronously by default, we need to wait for operations to complete if we want to refer to fields/objects that were sent to Neptune earlier in the same code.  
Read about the `wait()` and `sync()` methods here: https://docs.neptune.ai/logging/wait_and_sync/

In [ ]:
model_version.wait()

## (Neptune) Promote best model to production

### (Neptune) Fetch current champion model

In [ ]:
with neptune.init_model(with_id=f"{project_key}-KER") as model:
    model_versions_df = model.fetch_model_versions_table().to_pandas()
model_versions_df

In [ ]:
production_models = model_versions_df[model_versions_df["sys/stage"] == "production"]["sys/id"]
# assert (
#     len(production_models) == 1
# ), f"Multiple model versions found in production: {production_models.values}"

In [ ]:
prod_model_id = production_models.values[0]
print(f"Current champion model: {prod_model_id}")

In [ ]:
npt_prod_model = neptune.init_model_version(with_id=prod_model_id)
npt_prod_model_params = npt_prod_model["params/model"].fetch()
prod_model = tf.keras.models.model_from_json(npt_prod_model["serialized_model"].fetch())

npt_prod_model["model_weights"].download()
prod_model.load_weights("model_weights.h5")

### (Neptune) Evaluate current model on lastest test data

#### (Neptune) Fetch data parameters from the current champion model to preserve data preprocessing

In [ ]:
prod_data_params = npt_prod_model["params/data"].fetch()

print(prod_data_params)

#### Preparing test data according to fetched data parameters

In [ ]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/test", batch_size=prod_data_params["batch_size"]
)


print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=prod_data_params["max_features"],
    output_mode="int",
    output_sequence_length=prod_data_params["sequence_length"],
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

# Vectorize the data.
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [ ]:
# Evaluate champion model using the model's original loss and optimizer, but the current metric
prod_model.compile(
    loss=npt_prod_model_params["loss"],
    optimizer=npt_prod_model_params["optimizer"],
    metrics=model_params["metrics"],
)

_, prod_model_acc = prod_model.evaluate(test_ds)

### (Neptune) If challenger model outperforms production model, promote it to production

In [ ]:
print(f"Champion model accuracy: {prod_model_acc}\nChallenger model accuracy: {curr_model_acc}")

if curr_model_acc > prod_model_acc:
    print("Promoting challenger to champion")
    npt_prod_model.change_stage("archived")
    model_version.change_stage("production")
else:
    print("Archiving challenger model")
    model_version.change_stage("archived")

## (Neptune) Stop tracking
When working in an interactive notebook environment, we need to explicitly stop all initialized neptune objects to prevent unnecessary monitoring.  
Read more about stopping neptune objects here: https://docs.neptune.ai/usage/best_practices/#stopping-runs-and-other-objects

In [ ]:
npt_prod_model.stop()
model_version.stop()
run.stop()
project.stop()